# Import

In [13]:
import sys

sys.path.append("../")
import datetime
import hashlib
import json

import pandas as pd
import requests

from vook_db_v4.config import ClientId, aff_id, pid, sid

# Global

In [25]:
REQ_URL_CATE = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"

WANT_ITEMS = [
    "itemCode",
    "itemName",
    "itemPrice",
    "itemUrl",
]
PLATFORM_ID = 2
BRAND = "リーバイス"
BRAND_ID = 1
ITEM = "デニム"
ITEM_ID = 1
LINE = "66前期"
LINE_ID = 1
START_AGE = 1974
END_AGE = 1977
AGES_ID = 1
RUN_TIME = datetime.datetime.today().strftime("%Y%m%d_%H%M%S")
INFO_GET_DATE = datetime.datetime.today()
TABLE_COLUMNS = [
    "product_id",
    "product_name",
    "platform_id",
    "ages_id",
    "brand_id",
    "item_id",
    "line_id",
    "price",
    "info_get_date",
    "status",
]

# Main

In [3]:
query = f"{BRAND} ヴィンテージ {ITEM} {LINE}"

In [4]:
params = {
    "appid": ClientId,
    "output": "json",
    "query": query,
    "sort": "-price",
    "affiliate_id": aff_id,
    "affiliate_type": "vc",
    "results": 100,  # NOTE: 100個ずつしか取得できない。
}

start_num = 1
step = 100
max_products = 1000
l_df = []
for inc in range(0, max_products, step):
    params["start"] = start_num + inc
    df = pd.DataFrame(columns=WANT_ITEMS)
    res = requests.get(url=REQ_URL_CATE, params=params)
    res_cd = res.status_code
    if res_cd != 200:
        print(f"Bad request")
        break
    else:
        res = json.loads(res.text)
        if len(res["hits"]) == 0:
            print("If the number of returned items is 0, the loop ends.")
        print(f"Get Data")
        l_hit = []
        for h in res["hits"]:
            l_hit.append(
                (
                    h["index"],
                    h["name"],
                    h["price"],
                    h["url"],
                )
            )
        df = pd.DataFrame(l_hit, columns=WANT_ITEMS)
        l_df.append(df)

Get Data
Get Data
If the number of returned items is 0, the loop ends.
Get Data
If the number of returned items is 0, the loop ends.
Get Data
If the number of returned items is 0, the loop ends.
Get Data
If the number of returned items is 0, the loop ends.
Get Data
If the number of returned items is 0, the loop ends.
Get Data
If the number of returned items is 0, the loop ends.
Get Data
If the number of returned items is 0, the loop ends.
Get Data
Bad request


In [7]:
lines_raw = pd.concat(l_df, axis=0, ignore_index=True)
lines_raw

,itemCode,itemName,itemPrice,itemUrl
0,1,リーバイス 70s 501 66前期 BIGE ストレート デニムパンツ W31 Levi'...,327800,https://ck.jp.ap.valuecommerce.com/servlet/ref...
1,2,Levi's リーバイス ヴィンテージ 501 66前期 BigE 両面BigE 刻印6 オ...,80817,https://ck.jp.ap.valuecommerce.com/servlet/ref...
2,3,銀座店 リーバイス SALE ヴィンテージ 501 66 前期 W31 デニム パンツ,79900,https://ck.jp.ap.valuecommerce.com/servlet/ref...
3,4,リーバイス 60s 70s 66前期 ビッグE ストレート デニムパンツ Levi's ヴィ...,74800,https://ck.jp.ap.valuecommerce.com/servlet/ref...
4,5,リーバイス 70s 66前期 501 ストレート デニムパンツ w32 Levi's 赤耳 ...,63580,https://ck.jp.ap.valuecommerce.com/servlet/ref...
...,...,...,...,...
152,153,古着 リーバイス Levi's リーバイス501 66前期 デニムショーツ ショートパンツ ...,6930,https://ck.jp.ap.valuecommerce.com/servlet/ref...
153,154,Levi's リーバイス 517 66前期 スモールe SCOVILLジップ 刻印16 リ...,6017,https://ck.jp.ap.valuecommerce.com/servlet/ref...
154,155,中古 70's LEVISリーバイス505/66前期/デニムパンツ/実寸W38,5478,https://ck.jp.ap.valuecommerce.com/servlet/ref...
155,156,【中古】Levi's Vintage リーバイス ヴィンテージ/ビンテージ 501 66前期...,4378,https://ck.jp.ap.valuecommerce.com/servlet/ref...


In [9]:
lines_raw.to_csv(
    f"../data/output/{RUN_TIME}_lines_raw_{BRAND}_{ITEM}_{LINE}.csv", index=False
)

In [26]:
product_id_list = []
product_name_list = []
platform_id_list = []
ages_id_list = []
brand_id_list = []
item_id_list = []
line_id_list = []
price_list = []
info_get_date_list = []
info_published_date_list = []
status_list = []
for _, row in lines_raw.iterrows():
    product_id_list.append(hashlib.md5(str(row["itemCode"]).encode()).hexdigest())
    product_name_list.append(row["itemName"])
    price_list.append(row["itemPrice"])
    # info_published_date_list = []  # TODO
    # status_list = []  # TODO
tmp_cols = ["product_id", "product_name", "price"]
tmp_df = pd.DataFrame(
    zip(product_id_list, product_name_list, price_list),
    columns=tmp_cols,
)
tmp_df["platform_id"] = PLATFORM_ID
tmp_df["ages_id"] = AGES_ID
tmp_df["brand_id"] = BRAND_ID
tmp_df["item_id"] = ITEM_ID
tmp_df["line_id"] = LINE_ID
tmp_df["info_get_date"] = INFO_GET_DATE
tmp_df["status"] = ""
tmp_df = tmp_df[TABLE_COLUMNS]
lines = tmp_df.copy()

In [29]:
lines.dtypes

product_id               object
product_name             object
platform_id               int64
ages_id                   int64
brand_id                  int64
item_id                   int64
line_id                   int64
price                     int64
info_get_date    datetime64[ns]
status                   object
dtype: object

In [30]:
lines.head()

,product_id,product_name,platform_id,ages_id,brand_id,item_id,line_id,price,info_get_date,status
0,c4ca4238a0b923820dcc509a6f75849b,リーバイス 70s 501 66前期 BIGE ストレート デニムパンツ W31 Levi'...,2,1,1,1,1,327800,2023-05-03 16:06:23.825003,
1,c81e728d9d4c2f636f067f89cc14862c,Levi's リーバイス ヴィンテージ 501 66前期 BigE 両面BigE 刻印6 オ...,2,1,1,1,1,80817,2023-05-03 16:06:23.825003,
2,eccbc87e4b5ce2fe28308fd9f2a7baf3,銀座店 リーバイス SALE ヴィンテージ 501 66 前期 W31 デニム パンツ,2,1,1,1,1,79900,2023-05-03 16:06:23.825003,
3,a87ff679a2f3e71d9181a67b7542122c,リーバイス 60s 70s 66前期 ビッグE ストレート デニムパンツ Levi's ヴィ...,2,1,1,1,1,74800,2023-05-03 16:06:23.825003,
4,e4da3b7fbbce2345d7772b0674a318d5,リーバイス 70s 66前期 501 ストレート デニムパンツ w32 Levi's 赤耳 ...,2,1,1,1,1,63580,2023-05-03 16:06:23.825003,


In [31]:
lines.to_csv(f"../data/output/{RUN_TIME}_products.csv", index=False)